# Red Light Camera Natural Experiment

Linear regression analysis of factors affecting red light crashes

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from modules.myfuncs import *
import warnings
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import sqlite3
# import dask
# import dask.dataframe as dd

warnings.filterwarnings('ignore')
pd.options.display.max_rows = 100

In [15]:
conn = create_connection('database/rlc2.db')  # function from myfuncs file
c = conn.cursor()

sqlite3 version: 2.6.0
connected to database/rlc2.db


In [16]:
#rlc_df = pd.read_sql_query('''SELECT * FROM daily_violations''', conn)
#ints_df = pd.read_sql_query('''SELECT * FROM intersection_chars''', conn)
#crash_df = pd.read_sql_query('''SELECT * FROM signal_crashes''', conn)


In [17]:
print(sql_fetch_tables(c, conn))



def table_info(c, conn):
    '''
    prints out all of the columns of every table in db
    c : cursor object
    conn : database connection object
    '''
    tables = c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print('\t-' + col)
        print()

table_info(c, conn)


[('intersection_chars',), ('cam_locations',), ('cam_startend',), ('daily_violations',), ('all_crashes',), ('hourly_congestion',), ('hourly_weather',), ('region_data',), ('intersection_cams',), ('signal_crashes',), ('all_hours',), ('int_startend',)]
intersection_chars
	-protected_turn
	-total_lanes
	-medians
	-exit
	-split
	-way
	-underpass
	-no_left
	-angled
	-triangle
	-one_way
	-turn_lanes
	-lat
	-long
	-rlc
	-intersection
	-daily_traffic

cam_locations
	-camera_id
	-intersection
	-address
	-violation_date
	-violations
	-latitude
	-longitude
	-month
	-day
	-weekday
	-year

cam_startend
	-camera_id
	-start
	-end

daily_violations
	-intersection
	-camera_id
	-address
	-violation_date
	-violations
	-latitude
	-longitude
	-month
	-day
	-weekday
	-year

all_crashes
	-crash_record_id
	-rd_no
	-crash_date
	-posted_speed_limit
	-traffic_control_device
	-device_condition
	-weather_condition
	-lighting_condition
	-first_crash_type
	-trafficway_type
	-alignment
	-roadway_surface_cond
	-road_def

## Query our data
I would like to get hourly intersection data with the following columns:


signal_crashes
	-crash_date
	-posted_speed_limit
	-device_condition
	-weather_condition
	-lighting_condition
	-first_crash_type
	-trafficway_type
	-alignment
	-roadway_surface_cond
	-road_defect
	-report_type
	-crash_type
	-damage
	-prim_contributory_cause
	-sec_contributory_cause
	-street_no
	-street_direction
	-street_name
	-most_severe_injury
	-injuries_total
	-injuries_fatal
	-injuries_incapacitating
	-injuries_non_incapacitating
	-injuries_reported_not_evident
	-injuries_no_indication
	-injuries_unknown
	-latitude
	-longitude
	-lane_cnt
	-work_zone_i
	-work_zone_type
	-workers_present_i
	-intersection
	-year
	-month
	-day
	-hour
	-region_id
	-time
	-weekday

hourly_weather
	-temp
	-rain_1h
	-rain_3h
	-snow_1h
	-snow_3h
	-time
	-year
	-month
	-day
	-hour
	-weekday

hourly_congestion
	-year
	-month
	-day
	-hour
	-region_id
	-bus_count
	-num_reads
	-speed
	-weekday
    
Tables will be queried to JOIN on year, month, day, hour

WHAT I REALLY NEED HERE
A single dataset with the following:
- daily intersection chars (all the stuff I entered)
- daily intersection crashes (injuries, deaths, number etc)
- daily wx (temp, precip mainly)
- daily congestion (might not need this)
- daily violations (already in the format)

In [18]:
# Approximately how many entries should I expect to be looking at after filtering my data
# 3 years, 4 months x 153 intersections
print((365.25*3 + 30*4) *153)


186009.75


## Query my db to get a DataFrame with crashes and intersection
We will be using the number of crashes and injuries to do some t-tests

In [19]:
# THis is for the times the cams were off

cam_df = pd.read_sql_query('''WITH v AS(SELECT d.year,
                                                d.month,
                                                d.day,
                                                d.intersection,
                                                dv.violations,
                                                dv.violation_date
                                               
                                            FROM all_hours as d
                                            
                                            LEFT JOIN daily_violations as dv
                                                ON d.year = dv.year
                                                AND d.month = dv.month
                                                AND d.day = dv.day
                                                AND d.intersection = dv.intersection
                                                
                                            ), 
                                    se AS(SELECT *
                                         FROM int_startend as se
                                    ),
                                    cr AS(SELECT *
                                         FROM signal_crashes
                                        )
                                        
                                    SELECT  v.*,
                                    
                                            se.start,
                                            se.end
                                    FROM v
                                    LEFT JOIN se
                                        ON se.intersection = v.intersection
                                    ''', conn)

crash_df = pd.read_sql_query('''SELECT * FROM signal_crashes''', conn)
                                        
violations_df = pd.read_sql_query('''SELECT * FROM daily_violations''', conn)  
days_df = pd.read_sql_query('''SELECT * FROM all_hours''', conn)  

In [20]:
days_df[days_df['intersection']=='111TH AND HALSTED']  # missing from all_hours??  Probably missing from int_df

,year,month,day,intersection


What does my cam_df look like?

In [21]:
cam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533202 entries, 0 to 533201
Data columns (total 8 columns):
year              533202 non-null int64
month             533202 non-null int64
day               533202 non-null int64
intersection      533202 non-null object
violations        465184 non-null float64
violation_date    465184 non-null object
start             490074 non-null object
end               490074 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 32.5+ MB


What does my crash_df look like?

In [22]:
crash_df[crash_df['intersection']=='115TH AND HALSTED'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 255 to 59447
Data columns (total 51 columns):
crash_record_id                  40 non-null object
rd_no                            40 non-null object
crash_date                       40 non-null object
posted_speed_limit               40 non-null object
traffic_control_device           40 non-null object
device_condition                 40 non-null object
weather_condition                40 non-null object
lighting_condition               40 non-null object
first_crash_type                 40 non-null object
trafficway_type                  40 non-null object
alignment                        40 non-null object
roadway_surface_cond             40 non-null object
road_defect                      40 non-null object
report_type                      37 non-null object
crash_type                       40 non-null object
damage                           40 non-null object
prim_contributory_cause          40 non-null object
sec_cont

#### change my dates to datetime objects
Need to do this so we can do some date calculations to see which entries had the cameras on or off.

In [23]:
cam_df[['start', 'end']] = cam_df[['start', 'end']].apply(pd.to_datetime)
crash_df['crash_date'] = crash_df['crash_date'].apply(pd.to_datetime)

#### Make sure all my intersections have start-end dates.  I was originally missing a few

In [24]:
cam_df.isna().sum()  # one entry each accident with no violations/vio_date.  Days with no

year                  0
month                 0
day                   0
intersection          0
violations        68018
violation_date    68018
start             43128
end               43128
dtype: int64

In [25]:
cam_df[cam_df['start'].isna()]['intersection'].unique()
cam_df.dropna(subset=['start'], inplace=True)  # drop the few that don't have start-end.  Just a problem with my handmade dataset.  Can fix later

## Merge my DataFrames
Combine my camera info and crash info into a single df.  
Keep all of the cam_df info since it has all the date info.  Will join on

In [26]:
pd.options.display.max_rows = 200
all_df = cam_df.merge(crash_df, how='left', on=['year', 'month', 'day', 'intersection'])

In [27]:
all_df['date'] = all_df.apply(lambda x: datetime(int(x.year), int(x.month), int(x.day)), axis=1)

In [28]:
print(len(cam_df)) # should be same as all if everything merged properly
print(len(crash_df))
print(len(all_df))

print(len(crash_df.intersection.unique()))  
print(len(cam_df.intersection.unique()))

print(set(crash_df.intersection.unique()) - set(cam_df.intersection.unique()))  # missing very first one ???


490074
60158
490433
182
158
{'31ST AND CALIFORNIA', None, 'ELSTON AND FOSTER', 'PULASKI AND MONTROSE', 'ASHLAND AND 63RD', 'ARCHER/NARRAGANSETT AND 55TH', '111TH AND HALSTED', 'HALSTED AND 63RD', 'VINCENNES AND 111TH', 'HALSTED AND 83RD', 'ASHLAND AND DIVERSEY', 'KIMBALL AND LINCOLN', 'GARFIELD AND ASHLAND', 'BLUE ISLAND AND DAMEN', 'WESTERN AND PRATT', 'WESTERN AND ARMITAGE', 'MADISON AND CENTRAL', 'ASHLAND AND ARCHER', '83RD AND STONY ISLAND', 'WESTERN AND 51ST', 'COTTAGE GROVE AND 95TH', 'JEFFERY AND 79TH', 'CORNELL DRIVE AND 57TH', 'ASHLAND AND 47TH', 'HARLEM AND NORTHWEST HWY'}


In [29]:
all_df = all_df[all_df['date']>=datetime(2017,9,1)]

In [30]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321212 entries, 1154 to 490432
Data columns (total 56 columns):
year                             321212 non-null int64
month                            321212 non-null int64
day                              321212 non-null int64
intersection                     321212 non-null object
violations                       304425 non-null float64
violation_date                   304425 non-null object
start                            321212 non-null datetime64[ns]
end                              321212 non-null datetime64[ns]
crash_record_id                  10928 non-null object
rd_no                            10928 non-null object
crash_date                       10928 non-null datetime64[ns]
posted_speed_limit               10928 non-null object
traffic_control_device           10928 non-null object
device_condition                 10928 non-null object
weather_condition                10928 non-null object
lighting_condition             

In [31]:
pd.options.display.max_columns = 1000
#all_df[all_df['year'].isna()]
#all_df.iloc[535444]
all_df.year.unique()

array([2017, 2018, 2019, 2020])

In [32]:
# need to determine if crash occurred in or outside of cam on dates

def rlc_state(start, end, my_date):

    if (end - my_date).days >= 0 and (my_date - start).days >= 0:
        return 1
    elif (my_date - end).days > 0:
        return 0
    elif (start - my_date).days > 0:
        return 0
    else:
        return None

all_df['rlc_on'] = all_df.apply(lambda x: rlc_state(x.start, x.end, x.date), axis=1)

In [33]:
all_df[all_df['intersection']=='115TH AND HALSTED']

,year,month,day,intersection,violations,violation_date,start,end,crash_record_id,rd_no,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,crash_type,damage,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,lane_cnt,intersection_related_i,hit_and_run_i,crash_date_est_i,work_zone_i,work_zone_type,workers_present_i,hour,region_id,time,weekday,date,rlc_on
1154,2017,9,1,115TH AND HALSTED,7.0,2017-09-01 00:00:00,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01,1
1155,2017,9,1,115TH AND HALSTED,11.0,2017-09-01 00:00:00,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01,1
1156,2017,9,2,115TH AND HALSTED,6.0,2017-09-02 00:00:00,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-02,1
1157,2017,9,2,115TH AND HALSTED,7.0,2017-09-02 00:00:00,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-02,1
1158,2017,9,3,115TH AND HALSTED,4.0,2017-09-03 00:00:00,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-03,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2392,2020,11,27,115TH AND HALSTED,NaN,None,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-27,0
2393,2020,11,28,115TH AND HALSTED,NaN,None,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-28,0
2394,2020,11,29,115TH AND HALSTED,NaN,None,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-29,0
2395,2020,11,30,115TH AND HALSTED,NaN,None,2016-01-02,2017-10-26,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-30,0


In [34]:
all_df[(all_df['intersection']=='115TH AND HALSTED') & (all_df['crash_date'].notnull())][['crash_date', 'start', 'end', 'rlc_on']]

,crash_date,start,end,rlc_on
1206,2017-09-27 08:00:00,2016-01-02,2017-10-26,1
1207,2017-09-27 08:00:00,2016-01-02,2017-10-26,1
1284,2017-11-15 07:30:00,2016-01-02,2017-10-26,0
1289,2017-11-20 17:42:00,2016-01-02,2017-10-26,0
1377,2018-02-16 20:15:00,2016-01-02,2017-10-26,0
1425,2018-04-05 15:24:00,2016-01-02,2017-10-26,0
1439,2018-04-19 13:50:00,2016-01-02,2017-10-26,0
1503,2018-06-22 17:40:00,2016-01-02,2017-10-26,0
1520,2018-07-09 22:21:00,2016-01-02,2017-10-26,0
1568,2018-08-26 12:00:00,2016-01-02,2017-10-26,0


In [35]:
all_on = all_df[all_df['rlc_on']==1]

In [36]:
all_off = all_df[all_df['rlc_on']==0]

Initial look at how cam on/off compares with count of crashes and percent of intersection days with crashes

In [37]:
print(all_on.crash_record_id.count(), len(all_on), all_on.crash_record_id.count()/len(all_on),)
print(all_off.crash_record_id.count(), len(all_off), all_off.crash_record_id.count()/len(all_off) )

10462 311123 0.033626572127422275
466 10089 0.04618891862424423


# My first try at t-test

In [38]:
on = all_on['crash_record_id'].notnull()
off = all_off['crash_record_id'].notnull()

In [39]:
on.value_counts()

False    300661
True      10462
Name: crash_record_id, dtype: int64

In [40]:
off.value_counts()

False    9623
True      466
Name: crash_record_id, dtype: int64

In [41]:
print(on.sum(), len(on))
print(on.sum() / len(on))


print(off.sum(), len(off))
print(off.sum() / len(off))

10462 311123
0.033626572127422275
466 10089
0.04618891862424423


In [42]:
on_rate = on.mean()
off_rate = off.mean()
expected_off = on_rate * len(off)


print('rlc on crash mean: {:.4f}'.format(on_rate))
print('rlc off crash mean: {:.4f}'.format(off_rate))
print()

print('Actual crashes with rlc off:', off.sum())
print('Expected crashes with rlc off: {:.0f}'.format(expected_off))


rlc on crash mean: 0.0336
rlc off crash mean: 0.0462

Actual crashes with rlc off: 466
Expected crashes with rlc off: 339


In [43]:
n = len(off)
p = on_rate
var = n * p * (1-p)
std = np.sqrt(var)

print(std)  # deviation from expected. We are way outside that

18.106639286121965


In [44]:

z_score = (off.sum() - expected_off)/std
print(z_score)

6.999726001256297


In [45]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)




print(p_val)


1.2823178475776726e-12


First indication shows that red light cameras are statistically significant in reducing crashes.

# Second try at t-test

It is possible that the previous AB test was misleading.  Only a small percentage of cameras actually have dates where they were turned on/off during the examined time period. 

In [46]:
#p = 1 - stats.t.cdf(t, df)


In [47]:
all_df.columns

Index(['year', 'month', 'day', 'intersection', 'violations', 'violation_date',
       'start', 'end', 'crash_record_id', 'rd_no', 'crash_date',
       'posted_speed_limit', 'traffic_control_device', 'device_condition',
       'weather_condition', 'lighting_condition', 'first_crash_type',
       'trafficway_type', 'alignment', 'roadway_surface_cond', 'road_defect',
       'report_type', 'crash_type', 'damage', 'prim_contributory_cause',
       'sec_contributory_cause', 'street_no', 'street_direction',
       'street_name', 'beat_of_occurrence', 'num_units', 'most_severe_injury',
       'injuries_total', 'injuries_fatal', 'injuries_incapacitating',
       'injuries_non_incapacitating', 'injuries_reported_not_evident',
       'injuries_no_indication', 'injuries_unknown', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude', 'lane_cnt',
       'intersection_related_i', 'hit_and_run_i', 'crash_date_est_i',
       'work_zone_i', 'work_zone_type', 'workers_present_

In [48]:
# get cams that have a start date within the investigated period or an end date within it.
select_cams = all_df[((all_df['start'] > datetime(2017,9,1)) & (all_df['end'] > datetime(2017,9,1)))
                     | ((all_df['end'] < datetime(2020, 12,25)) & (all_df['end'] > datetime(2017, 9,1)))]

print(select_cams.intersection.unique())  # 9  total

['115TH AND HALSTED' 'CENTRAL AND MILWAUKEE'
 'FOSTER AND NORTHWEST HIGHWAY' 'HALSTED AND 103RD'
 'LAKE AND UPPER WACKER' 'MICHIGAN AND JACKSON' 'MICHIGAN AND ONTARIO'
 'MILWAUKEE AND CENTRAL' 'NORTHWEST HIGHWAY AND FOSTER']


In [49]:
select_cams.isna().sum()


year                                 0
month                                0
day                                  0
intersection                         0
violations                        3776
violation_date                    3776
start                                0
end                                  0
crash_record_id                  13231
rd_no                            13231
crash_date                       13231
posted_speed_limit               13231
traffic_control_device           13231
device_condition                 13231
weather_condition                13231
lighting_condition               13231
first_crash_type                 13231
trafficway_type                  13231
alignment                        13231
roadway_surface_cond             13231
road_defect                      13231
report_type                      13241
crash_type                       13231
damage                           13231
prim_contributory_cause          13231
sec_contributory_cause   

In [50]:
experiment = select_cams[select_cams['rlc_on']==0]
control = select_cams[select_cams['rlc_on']==1]

In [51]:
print(len(control), len(experiment))

10598 2941


In [52]:
n_experiment = len(experiment)
crash_experiment = experiment['crash_record_id'].notnull()

n_control = len(control)
crash_control = control['crash_record_id'].notnull()

In [53]:
print(crash_experiment.sum(), n_experiment)
print(crash_control.sum(), n_control)

106 2941
202 10598


In [54]:
on_rate = crash_control.mean()
off_rate = crash_experiment.mean()
expected_off = on_rate * len(experiment)


print('rlc on crash mean: {:.4f}'.format(on_rate))
print('rlc off crash mean: {:.4f}'.format(off_rate))
print()

print('Actual crashes with rlc off:', off.sum())
print('Expected crashes with rlc off: {:.0f}'.format(expected_off))

rlc on crash mean: 0.0191
rlc off crash mean: 0.0360

Actual crashes with rlc off: 466
Expected crashes with rlc off: 56


In [55]:
n = len(off)
p = on_rate
var = n * p * (1-p)
std = np.sqrt(var)

print(std)  # deviation from expected. We are way outside that

13.734377052015626


In [56]:
z_score = (off.sum() - expected_off)/std
print(z_score)

29.84801932671824


In [57]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)




print(p_val)

4.657007262706185e-196


### This gives a very different result.  Comparing only cams switched on/off.
Having a cam on is statistically significant for this group.

There could be another effect.  What if the camera that had the most down time also had more crashes.  That would skew the results dramatically

I believe the only real way to test this is by going through every camera of interest.



In [58]:
select_cams.intersection.unique()


array(['115TH AND HALSTED', 'CENTRAL AND MILWAUKEE',
       'FOSTER AND NORTHWEST HIGHWAY', 'HALSTED AND 103RD',
       'LAKE AND UPPER WACKER', 'MICHIGAN AND JACKSON',
       'MICHIGAN AND ONTARIO', 'MILWAUKEE AND CENTRAL',
       'NORTHWEST HIGHWAY AND FOSTER'], dtype=object)

We have 9 intersections of interest.  

In [59]:
# Above is a list of the cams in question.

In [136]:
def calculate_pval(intersection, control, experiment):
    # control is on, experiment is off
    control = control[control['intersection']==intersection]
    experiment = experiment[experiment['intersection']==intersection]

    
    n_experiment = len(experiment)
    crash_experiment = experiment['crash_record_id'].notnull()

    n_control = len(control)
    crash_control = control['crash_record_id'].notnull()
    
    on_rate = crash_control.mean()
    off_rate = crash_experiment.mean()
    expected_off = on_rate * n_experiment
    
    print()
    print('*'*20)
    print(intersection)
    print('Days on: {}  Days off: {}'.format(n_control, n_experiment))
    
    print('rlc on crash mean: {:.4f}'.format(on_rate))
    print('rlc off crash mean: {:.4f}'.format(off_rate))
    print()

    print('Actual crashes with rlc off:', crash_experiment.sum())
    print('Expected crashes with rlc off: {:.0f}'.format(expected_off))
    
    n = len(off)
    p = on_rate
    var = n * p * (1-p)
    std = np.sqrt(var)
    
    print()
    print('SD: {:.2f}'.format(std))  # deviation from expected. We are way outside that
    z_score = (off.sum() - expected_off)/std
    print('z score: {:.2f}'.format(z_score))
    
    p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
    print('p_value:', p_val)
    
    if p_val > 0.05: 
        print("Null Hypothesis Not Rejected")
    else:
        print('Null Hypothesis Rejected')

        
    
for intersection in select_cams.intersection.unique():
    calculate_pval(intersection, control, experiment)


********************
115TH AND HALSTED
Days on: 111  Days off: 1132
rlc on crash mean: 0.0180
rlc off crash mean: 0.0318

Actual crashes with rlc off: 36
Expected crashes with rlc off: 20

SD: 13.36
z score: 33.35
p_value: 3.429140034832806e-244
Null Hypothesis Rejected

********************
CENTRAL AND MILWAUKEE
Days on: 1134  Days off: 54
rlc on crash mean: 0.0097
rlc off crash mean: 0.0185

Actual crashes with rlc off: 1
Expected crashes with rlc off: 1

SD: 9.84
z score: 47.28
p_value: 0.0
Null Hypothesis Rejected

********************
FOSTER AND NORTHWEST HIGHWAY
Days on: 1133  Days off: 55
rlc on crash mean: 0.0088
rlc off crash mean: 0.0000

Actual crashes with rlc off: 0
Expected crashes with rlc off: 0

SD: 9.39
z score: 49.55
p_value: 0.0
Null Hypothesis Rejected

********************
HALSTED AND 103RD
Days on: 107  Days off: 1133
rlc on crash mean: 0.0280
rlc off crash mean: 0.0388

Actual crashes with rlc off: 44
Expected crashes with rlc off: 32

SD: 16.58
z score: 26.19


## Build out a balanced experiment and control
We have an imbalance. Each intersection is imbalanced cam_on/cam_off.  We are unable to use all of the data for this reason. If we lump all of the intersections together, the cams that were primarily on or primarily off will be overrepresented in their respective groups.  

I propose balancing every intersection.  If I only have 100 days with the camera off, I will only take 100 days of data from the camera on data.  We should also control for time of year if possible.  We could take random samples perhaps to minimize this effect. 

In [137]:
np.random.seed(42)

all_ints_on = pd.Series()
all_ints_off = pd.Series()


for myint in select_cams['intersection'].unique():
    myint_df = select_cams[select_cams['intersection']==myint]  # get all days regardless of on/off status
    cam_on = myint_df[myint_df.rlc_on==1]
    cam_off = myint_df[myint_df.rlc_on==0]
    cam_on = cam_on.sample(frac=1).reset_index(drop=True)
    cam_off = cam_off.sample(frac=1).reset_index(drop=True)

    
    balanced_n = min(cam_on.year.count(), cam_off.year.count())  # get the smaller size (control or experiment)
    if balanced_n:
        print('{:>30}: {}'.format(myint, balanced_n))
    
    # use balanced_n to add same amount to each series
    all_ints_on = all_ints_on.append(cam_on.iloc[:balanced_n]['crash_record_id'])
    all_ints_off = all_ints_off.append(cam_off.iloc[:balanced_n]['crash_record_id'])

print()
print(len(all_ints_on), 'days of cam on, and', len(all_ints_off), 'days of cam off.')
print(all_ints_on.count(), all_ints_off.count())

             115TH AND HALSTED: 111
         CENTRAL AND MILWAUKEE: 54
  FOSTER AND NORTHWEST HIGHWAY: 55
             HALSTED AND 103RD: 107
         LAKE AND UPPER WACKER: 173
          MICHIGAN AND JACKSON: 157
          MICHIGAN AND ONTARIO: 157
         MILWAUKEE AND CENTRAL: 54
  NORTHWEST HIGHWAY AND FOSTER: 26

894 days of cam on, and 894 days of cam off.
17 34


In [140]:
# after all that, we only have 894 in each dataset for the win.  We only have 9 cameras represented in the set.

In [144]:
# pulled info from coin flip analysis lab in mod 2 t-test


def calc_pval(control, experiment):
    # control is on, experiment is off
    #control = control[control['intersection']==intersection]
    #experiment = experiment[experiment['intersection']==intersection]

    n_experiment = len(experiment)
    crash_experiment = experiment.notnull()

    n_control = len(control)
    crash_control = control.notnull()
    
    on_rate = crash_control.mean()
    off_rate = crash_experiment.mean()
    expected_off = on_rate * n_experiment
    
    print('n days total: {}'.format(n_control + n_experiment))
    print('Total crashes in set: {}\n'.format(crash_control.sum() + crash_experiment.sum()))
    
    print('Camera ON mean crashes per day: {:.4f}'.format(on_rate))
    print('Camera OFF mean crashes per day: {:.4f}'.format(off_rate))
    print()

    print('Actual crashes with rlc off:', crash_experiment.sum())
    print('Expected crashes with rlc off: {:.0f}'.format(expected_off))
    
    n = len(off) # n for my experiment
    p = on_rate # mean for my control (expected mean for my experiment)
    var = n * p * (1-p)
    std = np.sqrt(var)
    
    print()
    print('std: {:.2f}'.format(std))  # deviation from expected. 
    z_score = (off.sum() - expected_off)/std
    print('z score: {:.2f}'.format(z_score))
    
    p_val = stats.norm.sf(z_score) # calc the pval from the zscore.
    print('p: {:.3e}'.format(p_val))
    print()
    
    if p_val < 0.05: 
        print('Null Hypothesis Rejected')
    else:
        print("Null Hypothesis Not Rejected")
        

calc_pval(all_ints_on, all_ints_off)
print('alpha:', 0.05)


n days total: 1788
Total crashes in set: 51

Camera ON mean crashes per day: 0.0190
Camera OFF mean crashes per day: 0.0380

Actual crashes with rlc off: 34
Expected crashes with rlc off: 17

std: 13.72
z score: 32.73
p: 3.000e-235

Null Hypothesis Rejected
alpha: 0.05


# Bootstrap resampling method with t-test

### Make my two balanced date series

In [145]:
# bootstrap resampling
np.random.seed(42)

all_ints_on = pd.Series()
all_ints_off = pd.Series()


for myint in select_cams['intersection'].unique():
    myint_df = select_cams[select_cams['intersection']==myint]  # get all days regardless of on/off status
    cam_on = myint_df[myint_df.rlc_on==1]
    cam_off = myint_df[myint_df.rlc_on==0]
    cam_on = cam_on.sample(frac=1).reset_index(drop=True)
    cam_off = cam_off.sample(frac=1).reset_index(drop=True)

    
    balanced_n = min(cam_on.year.count(), cam_off.year.count())  # get the smaller size (control or experiment)
    if balanced_n:
        print('{:>30}: {}'.format(myint, balanced_n))
    
    # use balanced_n to add same amount to each series
    all_ints_on = all_ints_on.append(cam_on.iloc[:balanced_n]['crash_record_id'])
    all_ints_off = all_ints_off.append(cam_off.iloc[:balanced_n]['crash_record_id'])

print()
print(len(all_ints_on), 'days of cam on, and', len(all_ints_off), 'days of cam off.')
print(all_ints_on.count(), all_ints_off.count())

             115TH AND HALSTED: 111
         CENTRAL AND MILWAUKEE: 54
  FOSTER AND NORTHWEST HIGHWAY: 55
             HALSTED AND 103RD: 107
         LAKE AND UPPER WACKER: 173
          MICHIGAN AND JACKSON: 157
          MICHIGAN AND ONTARIO: 157
         MILWAUKEE AND CENTRAL: 54
  NORTHWEST HIGHWAY AND FOSTER: 26

894 days of cam on, and 894 days of cam off.
17 34


In [146]:
def bootstrap(A, B):
    universe = list(A) + list(B)
    universe_shuffled = np.random.choice(universe, size=len(universe), replace=True)
    new_a = universe_shuffled[:len(A)]
    new_b = universe_shuffled[len(A):]
    return new_a, new_b

In [147]:
# bootstrap resampling
np.random.seed(42)

all_ints_on = pd.Series()
all_ints_off = pd.Series()


for myint in select_cams['intersection'].unique():
    myint_df = select_cams[select_cams['intersection']==myint]  # get all days regardless of on/off status
    cam_on = myint_df[myint_df.rlc_on==1]
    cam_off = myint_df[myint_df.rlc_on==0]
    cam_on = cam_on.sample(frac=1).reset_index(drop=True)
    cam_off = cam_off.sample(frac=1).reset_index(drop=True)

    
    balanced_n = min(cam_on.year.count(), cam_off.year.count())  # get the smaller size (control or experiment)
    if balanced_n:
        print('{:>30}: {}'.format(myint, balanced_n))
    
    # use balanced_n to add same amount to each series
    all_ints_on = all_ints_on.append(cam_on.iloc[:balanced_n]['crash_record_id'])
    all_ints_off = all_ints_off.append(cam_off.iloc[:balanced_n]['crash_record_id'])

print()
print(len(all_ints_on), 'days of cam on, and', len(all_ints_off), 'days of cam off.')
print(all_ints_on.count(), all_ints_off.count())

             115TH AND HALSTED: 111
         CENTRAL AND MILWAUKEE: 54
  FOSTER AND NORTHWEST HIGHWAY: 55
             HALSTED AND 103RD: 107
         LAKE AND UPPER WACKER: 173
          MICHIGAN AND JACKSON: 157
          MICHIGAN AND ONTARIO: 157
         MILWAUKEE AND CENTRAL: 54
  NORTHWEST HIGHWAY AND FOSTER: 26

894 days of cam on, and 894 days of cam off.
17 34


In [148]:
exp = all_ints_off.notnull()  # series of True False, did crash occur
con = all_ints_on.notnull()


# bootstrap iterations
iterations = 10**4
diff_mu_a_b = np.mean(exp) - np.mean(con)
num = 0 #Initialize numerator



for n in range(iterations):
    ai, bi = bootstrap(exp, con)
    diff_mu_ai_bi = np.mean(ai) - np.mean(bi)
    if diff_mu_ai_bi >= diff_mu_a_b:
        num +=1
p_value = num / iterations
print('P-value: {}'.format(p_value))

P-value: 0.0093


Using the bootstrap resampling method with 10k iterations, we are able to get a very high confidence that the presence of red light cameras reduced crashes at the 9 intersections that were turned on/off.

### Evaluate the power of my t-test. 
I know the number of samples and the means, so I should be able to find the power.
Reminder: power tells us how likely it is that we reject a null hypothesis when it is incorrect.

In [149]:
from statsmodels.stats.power import TTestIndPower, TTestPower

power_analysis = TTestIndPower()

# total days of observations
observations = len(con) + len(exp)
print('Observations: {}'.format(observations))

# effect size is measure of effect (small 02, medium )
effect_size = abs(exp.mean() - con.mean())/ con.std()
print('Effect size: {:.3f}'.format(effect_size) ) # effect size
power = power_analysis.solve_power(effect_size=effect_size, nobs1=observations, alpha=.05)  # nobs1, alpha, power, effect_size
print('Power: {:.3f}'.format(power))

Observations: 1788
Effect size: 0.139
Power: 0.986


The power calculation makes me comfortable that we could even lower the alpha, since we are unlikely to mistakenly reject the null hypothesis when correct.

# Same bootstrap resampling t-test with injuries 

In [150]:
# bootstrap resampling
np.random.seed(42)

all_ints_on = pd.Series()
all_ints_off = pd.Series()


for myint in select_cams['intersection'].unique():
    myint_df = select_cams[select_cams['intersection']==myint]  # get all days regardless of on/off status
    cam_on = myint_df[myint_df.rlc_on==1]
    cam_off = myint_df[myint_df.rlc_on==0]
    cam_on = cam_on.sample(frac=1).reset_index()
    cam_off = cam_off.sample(frac=1).reset_index()

    
    balanced_n = min(cam_on.year.count(), cam_off.year.count())  # get the smaller size (control or experiment)
    if balanced_n:
        print('{:>30}: {}'.format(myint, balanced_n))
    
    # use balanced_n to add same amount to each series
    all_ints_on = all_ints_on.append(cam_on.iloc[:balanced_n]['injuries_total'])
    all_ints_off = all_ints_off.append(cam_off.iloc[:balanced_n]['injuries_total'])

print()
print(len(all_ints_on), 'days of cam on, and', len(all_ints_off), 'days of cam off.')
print(all_ints_on.count(), all_ints_off.count())

             115TH AND HALSTED: 111
         CENTRAL AND MILWAUKEE: 54
  FOSTER AND NORTHWEST HIGHWAY: 55
             HALSTED AND 103RD: 107
         LAKE AND UPPER WACKER: 173
          MICHIGAN AND JACKSON: 157
          MICHIGAN AND ONTARIO: 157
         MILWAUKEE AND CENTRAL: 54
  NORTHWEST HIGHWAY AND FOSTER: 26

894 days of cam on, and 894 days of cam off.
17 34


In [151]:
exp = all_ints_off.fillna(0)
exp = exp.astype(int)
con = all_ints_on.fillna(0)
con= con.astype(int)

In [152]:
print('Injuries with cams on:  {}'.format(con.sum()))
print('Injuries with cams off: {}'.format(exp.sum()))

Injuries with cams on:  6
Injuries with cams off: 12


In [153]:
# bootstrap iterations
iterations = 10**4
diff_mu_a_b = np.mean(exp) - np.mean(con)
num = 0 #Initialize numerator



for n in range(iterations):
    ai, bi = bootstrap(con, exp)
    diff_mu_ai_bi = np.mean(ai) - np.mean(bi)
    if diff_mu_ai_bi >= diff_mu_a_b:
        num +=1
p_value = num / iterations
print('P-value: {}'.format(p_value))

P-value: 0.1354


## Check the results with a built in function

Results here are inconclusisive.  We cannot reject the null hypothesis that red light cameras have no affect on the number of injuries.
It should be noted that there are only 10 and 9 injuries in the dataset.  More data might yield a different result.

In [154]:
from scipy import stats
# ttest from libarary to doublecheck work

In [155]:
tStat, pValue = stats.ttest_ind(exp, con, equal_var = False) #run independent sample T-Test
print("P-Value: {:.3f}   T-Statistic: {:.3f}".format(pValue,tStat)) #print the P-Value and the T-Statistic


P-Value: 0.272   T-Statistic: 1.099


## T-test for dangerous crash types
#### Same thing now with types of crashes we are concerned with.  


Start by building a balanced dataset

In [156]:
# bootstrap resampling
np.random.seed(337)

all_ints_on = pd.Series()
all_ints_off = pd.Series()


for myint in select_cams['intersection'].unique():
    myint_df = select_cams[select_cams['intersection']==myint]  # get all days regardless of on/off status
    cam_on = myint_df[myint_df.rlc_on==1]
    cam_off = myint_df[myint_df.rlc_on==0]
    cam_on = cam_on.sample(frac=1).reset_index(drop=True)
    cam_off = cam_off.sample(frac=1).reset_index(drop=True)

    
    balanced_n = min(cam_on.year.count(), cam_off.year.count())  # get the smaller size (control or experiment)
    if balanced_n:
        print('{:>30}: {}'.format(myint, balanced_n))
    
    # use balanced_n to add same amount to each series
    all_ints_on = all_ints_on.append(cam_on.iloc[:balanced_n]['first_crash_type'])
    all_ints_off = all_ints_off.append(cam_off.iloc[:balanced_n]['first_crash_type'])

print()
print(len(all_ints_on), 'days of cam on, and', len(all_ints_off), 'days of cam off.')
print(all_ints_on.count(), all_ints_off.count())

             115TH AND HALSTED: 111
         CENTRAL AND MILWAUKEE: 54
  FOSTER AND NORTHWEST HIGHWAY: 55
             HALSTED AND 103RD: 107
         LAKE AND UPPER WACKER: 173
          MICHIGAN AND JACKSON: 157
          MICHIGAN AND ONTARIO: 157
         MILWAUKEE AND CENTRAL: 54
  NORTHWEST HIGHWAY AND FOSTER: 26

894 days of cam on, and 894 days of cam off.
18 34


In [157]:
all_ints_on.unique()

array([nan, 'TURNING', 'PEDESTRIAN', 'SIDESWIPE SAME DIRECTION', 'ANGLE',
       'PEDALCYCLIST', 'REAR END'], dtype=object)

In [158]:
dangerous = ['REAR END']  # chicago website said this is the one they were trying to prevent.

def targeted_crash_count(crash_type):
    if crash_type in dangerous:
        return 1
    return 0

con = all_ints_on.apply(targeted_crash_count)
exp = all_ints_off.apply(targeted_crash_count)

In [159]:
print(con.sum(), exp.sum()) # This will not be enough to be conclusive

1 7


In [160]:
print('Targeted crashes with cams on:  {}'.format(con.sum()))
print('Targeted crashes with cams off: {}'.format(exp.sum()))

Targeted crashes with cams on:  1
Targeted crashes with cams off: 7


In [161]:
# bootstrap iterations
iterations = 10**4
diff_mu_a_b = np.mean(exp) - np.mean(con)
num = 0 #Initialize numerator



for n in range(iterations):
    ai, bi = bootstrap(con, exp)
    diff_mu_ai_bi = np.mean(ai) - np.mean(bi)
    if diff_mu_ai_bi >= diff_mu_a_b:
        num +=1
p_value = num / iterations
print('P-value: {}'.format(p_value))

P-value: 0.0161


Small sample size.  The event was infrequent at best.

In [162]:
tStat, pValue = stats.ttest_ind(exp, con, equal_var = False) #run independent sample T-Test
print("P-Value: {:.3f}   T-Statistic: {:.3f}".format(pValue,tStat)) #print the P-Value and the T-Statistic


P-Value: 0.034   T-Statistic: 2.128


### Power analysis of t-test

In [163]:
from statsmodels.stats.power import TTestIndPower, TTestPower

power_analysis = TTestIndPower()

In [164]:

effect_size = abs(con.mean() - exp.mean())/ exp.std()
print('Effect size:', effect_size)  # effect size


# test has low power, and is unlikely to reject null hypothesis
power = power_analysis.solve_power(effect_size=.01, nobs1=1600, alpha=.05)  # nobs1, alpha, power, effect_size
print('Power:', power )

Effect size: 0.07610223132364424
Power: 0.059208464478414775


# T-test for Rear-end accidents

In [165]:
crash_df.head()

,crash_record_id,rd_no,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,crash_type,damage,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,lane_cnt,intersection_related_i,hit_and_run_i,crash_date_est_i,work_zone_i,work_zone_type,workers_present_i,intersection,year,month,day,hour,region_id,time,weekday
0,00e47f189660cd8ba1e85fc63061bf1d8465184393f134...,JC194776,2019-03-21 22:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,8301,S,CICERO AVE,834,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,22,5,3,41.741804,-87.740954,4,Y,None,None,None,None,None,None,2019,3,21,22,22,2019-03-21 22:50:00,3
1,0209e21f298984f7375742b7ef27c9880b485f41123a12...,JB415436,2018-08-30 17:45:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,"OVER $1,500",IMPROPER OVERTAKING/PASSING,IMPROPER LANE USAGE,600,W,DIVISION ST,1822,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,17,5,8,41.903825,-87.643286,None,Y,None,None,None,None,None,None,2018,8,30,17,12,2018-08-30 17:45:00,3
2,0211e1f766f3940dfa87375661d25b716655e908c320cc...,JC301403,2019-06-11 08:40:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,"$501 - $1,500",UNABLE TO DETERMINE,NOT APPLICABLE,50,E,GARFIELD BLVD,225,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,8,3,6,41.794779,-87.623828,None,Y,None,None,None,None,None,None,2019,6,11,8,20,2019-06-11 08:40:00,1
3,f5464038ed524b7229878b786bcebdb35afaed6ce0a201...,JD280192,2020-06-29 14:49:00,30,TRAFFIC SIGNAL,NOT FUNCTIONING,CLEAR,DAYLIGHT,ANGLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,2600,S,CALIFORNIA BLVD,1034,2,NONINCAPACITATING INJURY,1,0,0,1,0,1,0,14,2,6,41.844608,-87.694805,None,Y,None,None,None,None,None,None,2020,6,29,14,14,2020-06-29 14:49:00,0
4,03c8fee8a0cb0d303e972a873228b444a47b7b1ed1e2d9...,JB317419,2018-06-22 07:25:00,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,UNABLE TO DETERMINE,8301,S,CICERO AVE,834,2,NONINCAPACITATING INJURY,2,0,0,2,0,2,0,7,6,6,41.741804,-87.740954,6,Y,None,None,None,None,None,None,2018,6,22,7,22,2018-06-22 07:25:00,4


In [166]:
cam_df.head()


,year,month,day,intersection,violations,violation_date,start,end
0,2016,1,1,115TH AND HALSTED,NaN,None,2016-01-02,2017-10-26
1,2016,1,2,115TH AND HALSTED,6.0,2016-01-02 00:00:00,2016-01-02,2017-10-26
2,2016,1,2,115TH AND HALSTED,14.0,2016-01-02 00:00:00,2016-01-02,2017-10-26
3,2016,1,3,115TH AND HALSTED,2.0,2016-01-03 00:00:00,2016-01-02,2017-10-26
4,2016,1,3,115TH AND HALSTED,11.0,2016-01-03 00:00:00,2016-01-02,2017-10-26


In [167]:
def rlc_state(start, end, crash):
    if (end - crash).days > 0 and (crash - start).days > 0:
        return 1
    elif (crash - end).days > 0:
        return 0
    elif (start - crash).days > 0:
        return 0
    else:
        return None

cam_df['rlc_on'] = cam_df.apply(lambda x: rlc_state(x.start, x.end, x.crash_date), axis=1)

AttributeError: ("'Series' object has no attribute 'crash_date'", 'occurred at index 0')

In [ ]:
cam_df.crash_date.unique()
print(nocam_df.crash_date.notnull().sum())
print(nocam_df.crash_date.isnull().sum())

In [ ]:
pd.options.display.max_rows = 1000
nocam_df[nocam_df.crash_date.notnull()]['crash_date'].describe()

In [ ]:
print(365*5*153)

In [ ]:
nocam_df.head()